## Notebook Purpose

analyze results for feature map regularization

- csv result files
    - `run_20201120-092851.csv`, `run_20201120-092903.csv`: search over all hparams on three file_ids (1000273, 1000325, 1000464), for 2000 iterations
        - via `search_hyperparam.py` / `train.py`
        - resulting best 20 trial_ids saved for future work in `trials_best_20201120.json`
    - check statistical significance...
        - `run_20201121-160658.csv`: used 20 best trial_ids over a larger set of file_ids for 2000 iterations
            - are three samples sufficient for statistically significant results?
            - via `search_hyperparam_larger_set.py` / `train_larger_set.py`
            - `run_20201123-065717.csv`: baseline (alpha_fm=0) for larger set of file_ids with 2000 iterations
        - `run_20201121-160718.csv`: used 20 best trial_ids over same set of three file_ids for 10000 iterations
            - are 2000 samples sufficient for statistically significant results?
            - via `search_hyperparam_orig_three.py` / `train_orig_three.py`
            - `run_20201123-065450.csv`: baseline (alpha_fm=0) for three file_ids with 10000 iterations
    - across 10000 iterations (`run_20201121-160718.csv` and `run_20201124-134955.csv`)
        - best when `iter_start_fm_loss=0`, `alpha_fm=0.00010`, `weight_method='all'`, `downsamp_method='bilinear'` w 'nearest' a close second
        - compared to baselines (`run_20201123-065450.csv` and `run_20201124-134811.csv`)
    - across 25000 iterations (`run_20201128-173303.csv` and `run_20201128-173259.csv`)
        - i.e. using best trial_id configs for 10000 iterations
        - both alpha_fm=0.0001 and alpha_fm=0 perform worse than for 10000 iterations, but gap is up to 2% improvement

## Results

run over 15 samples, given configs specified above. presented as {ssim_dc, psnr_dc, ssim_est, psnr_est}

- 25000 iterations
    - w/ fm: {.7411, 31.4083, .6595, 29.9445}, trial_id=udt0c920
    - no fm: {.7261, 31.1230, .6518, 29.7562}, trial_id=0000_25k
- 10000 iterations
    - w/ fm: {.7508, 30.4503, .5653, 27.5485}, trial_id=autybby9
    - no fm: {.7387, 29.9558, .5498, 27.0144}, trial_id=0000_10k
        - TODO: compute ssim_est, psnr_est
- 2000 iterations
    - w/ fm: {.6933, 27.6724, .4379, 22.8779}, trial_id=jlpxw8ir
    - no fm: {.6895, 27.5732, x, x}, trial_id=00000_2k
- note: params were tuned on the test set. should treat that as a validation set, then redo this w set params on a test set

In [109]:
import os, sys
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import json

### load original csvs

### aggregate trial_ids into same column, i.e. average ssim/psnr across file_ids

In [233]:
display_rows = 10#len(df)
pd.set_option('display.max_rows', display_rows)

def agg_sort_df(df_in):
    ''' given df, aggregate trial_ids across means for different file_ids
        sort df rows by the highest ssim_dc '''
    try:
        agg_fns = {'ssim_dc': 'mean', 'psnr_dc': 'mean', 'ssim_est':'mean', 'psnr_est':'mean', \
           'alpha_fm': 'first', 'num_iter': 'first', 'iter_start_fm_loss': 'first', \
           'weight_method': 'first', 'downsamp_method':'first'}
        df = df_in.groupby(df_in['trial_id']).aggregate(agg_fns).reset_index()
    except:
        agg_fns = {'ssim_dc': 'mean', 'psnr_dc': 'mean', 'ssim_est':'mean', 'psnr_est':'mean', \
           'alpha_fm': 'first', 'num_iter': 'first', \
           'weight_method': 'first', 'downsamp_method':'first'}
        df = df_in.groupby(df_in['trial_id']).aggregate(agg_fns).reset_index()
    df = df.sort_values(by='ssim_dc', ascending=False)
    
    return df

In [194]:
path_results = '/home/vanveen/ConvDecoder/expmt/results/'
# csv_files = [f for f in os.listdir(path_results) if isfile(join(path_results, f))]
csv_files = ['run_20201120-092851.csv', 'run_20201120-092903.csv']
csv_paths = [path_results + f for f in csv_files]
df_in = pd.concat((pd.read_csv(f) for f in csv_paths))

df = agg_sort_df(df_in)
df

,trial_id,ssim_dc,psnr_dc,ssim_est,psnr_est,alpha_fm,num_iter,iter_start_fm_loss,weight_method,downsamp_method
66,f1g4rdh9,0.728433,28.932900,0.502467,24.481000,0.00010,2000,0,late,nearest
77,i4csenaj,0.727133,28.824767,0.501000,24.403667,0.00010,2000,0,late,bicubic
61,dyn9rycz,0.726833,28.816833,0.496467,24.212033,0.00100,2000,1000,late,bilinear
139,w1v8v9ni,0.726333,28.807200,0.500367,24.465900,0.00001,2000,0,all,bilinear
12,1p0kn9i7,0.725833,28.754067,0.492600,24.087367,0.00010,2000,1600,early,nearest
...,...,...,...,...,...,...,...,...,...,...
128,ui2gux4g,0.649633,26.584367,0.441233,22.162600,0.00100,2000,0,early,bilinear
93,kg9zpja1,0.642900,26.635567,0.413400,20.919067,0.00100,2000,0,all,bicubic
1,0b8qud3y,0.639467,26.834233,0.424267,21.567500,0.00100,2000,0,all,bilinear
30,7jlwlalt,0.634600,25.516700,0.406167,18.853367,0.01000,2000,1000,early,nearest


### confirmed: variance is lowest for `alpha_fm=0`, since that negates all other variables

- TODO: in future, don't need to run this over all variables, since it's the same experiment

### get best value for each variable

finding: given std() of each individual variable, too difficult to say whether one value performs better than another with confidence. also many other variables at play

In [226]:
def get_1d_results(df, variable=None):
    '''return results slicing across each individual dimension'''
    
    if not variable:
        variable_list = ['alpha_fm', 'iter_start_fm_loss', 'weight_method', 'downsamp_method']
    else:
        variable_list = [variable]

    for var in variable_list:

        print(var)
        var_values = list(set(df[var].tolist()))
        var_values.sort()

        for value in var_values:

            df_val = df.loc[df[var] == value]
            print('{}: ssim ~ N({}, {}), psnr ~ N({}, {}), n={}'.format(value, \
                         np.round(df_val['ssim_dc'].mean(), 4), \
                         np.round(df_val['ssim_dc'].std(), 4), \
                         np.round(df_val['psnr_dc'].mean(), 4), \
                         np.round(df_val['psnr_dc'].std(), 4), len(df_val)))

        print('')

### save dictionary for best 20 trial_ids 

then more thoroughly analyze these trial_ids, e.g. across >3 samples or >2000 iterations

In [128]:
def save_df_to_json(json_path, df):
    ''' given dataframe, save dict version to json '''
    
    dict_ = df.to_dict('index')
    
    with open(json_path, 'w') as f:
        json.dump(dict_, f)
        
    return dict_  

json_path = path_results + 'trials_best_20201120.json'

# dict_best_results = save_df_to_json(df[:20])

### get baseline i.e. `alpha_fm=0` for iter 2000 and 10000

In [219]:
df_alpha0_2k = pd.read_csv(path_results + 'run_20201123-065717.csv')

csv_files_alpha0_10k = ['run_20201123-065450.csv', 'run_20201124-134811.csv']
csv_paths = [path_results + f for f in csv_files_alpha0_10k]
df_alpha0_10k = pd.concat((pd.read_csv(f) for f in csv_paths))

def print_avg_std(df):
    print('ssim ~ N({}, {}), psnr ~ N({}, {})'.format(\
                 np.round(df['ssim_dc'].mean(), 4), \
                 np.round(df['ssim_dc'].std(), 4), \
                 np.round(df['psnr_dc'].mean(), 4), \
                 np.round(df['psnr_dc'].std(), 4)))

print('2000 iterations across all samples')
print_avg_std(df_alpha0_2k)
print('')
print('10000 iterations across three samples')
df_alpha0_10k = df_alpha0_10k.drop_duplicates('file_id')
print_avg_std(df_alpha0_10k)

2000 iterations across all samples
ssim ~ N(0.6895, 0.081), psnr ~ N(27.5732, 2.0144)

10000 iterations across three samples
ssim ~ N(0.7387, 0.0842), psnr ~ N(29.9558, 2.2186)


### iter 2000 across all samples

In [221]:
df_2k_ = pd.read_csv(path_results + 'run_20201121-160658.csv')

# must append original three samples w matching trial_id
top_trials = list(set(df_2k_['trial_id'].tolist()))
df_2k_3 = df_in[df_in['trial_id'].isin(top_trials)]

df_2k_all = pd.concat([df_2k_, df_2k_3])
df_2k_all = agg_sort_df(df_2k_all)
df_2k_all

,trial_id,ssim_dc,psnr_dc,ssim_est,psnr_est,alpha_fm,num_iter,iter_start_fm_loss,weight_method,downsamp_method
6,f1g4rdh9,0.693673,27.742191,0.443164,23.114291,0.00010,2000,0,late,nearest
19,w1v8v9ni,0.693491,27.701418,0.445564,23.136845,0.00001,2000,0,all,bilinear
9,jlpxw8ir,0.693291,27.672355,0.437873,22.877909,0.00010,2000,0,all,bilinear
13,r71wfg6k,0.692518,27.619409,0.438836,22.863400,0.00001,2000,0,all,nearest
7,i4csenaj,0.692382,27.649891,0.435355,22.847491,0.00010,2000,0,late,bicubic
...,...,...,...,...,...,...,...,...,...,...
8,ixn1eec3,0.691227,27.564955,0.428418,22.590864,0.00010,2000,1600,all,bilinear
1,39n9u56h,0.691164,27.609009,0.431609,22.759145,0.00100,2000,1000,late,bicubic
10,k6aoqok1,0.691155,27.601473,0.434109,22.851518,0.00001,2000,1000,early,bicubic
17,vz8mku1t,0.691064,27.571391,0.432873,22.744473,0.00100,2000,0,late,nearest


### iter 10000 across all samples

In [222]:
csv_files_10k = ['run_20201121-160718.csv', 'run_20201124-134955.csv']
csv_paths = [path_results + f for f in csv_files_10k]
df_10k = pd.concat((pd.read_csv(f) for f in csv_paths))

# save_df_to_json(path_results + 'trials_best_20201121-160718.json', df_10k)

df_10k = agg_sort_df(df_10k)
df_10k

,trial_id,ssim_dc,psnr_dc,ssim_est,psnr_est,alpha_fm,num_iter,iter_start_fm_loss,weight_method,downsamp_method
8,lwd6legj,0.750927,30.515018,0.570364,27.690282,0.00001,10000,0,all,nearest
4,autybby9,0.750800,30.450264,0.565327,27.548536,0.00010,10000,0,all,bilinear
2,6efqyxuy,0.749564,30.377364,0.563818,27.528309,0.00010,10000,1000,early,bilinear
1,409jkpvb,0.748027,30.322673,0.563945,27.429736,0.00010,10000,0,late,bicubic
17,ymbp19bo,0.747518,30.391000,0.567127,27.527636,0.00001,10000,0,early,bilinear
...,...,...,...,...,...,...,...,...,...,...
13,riz67c4j,0.741645,30.096936,0.554518,27.184000,0.00100,10000,1000,late,bilinear
6,hgejliq8,0.741418,30.097100,0.553282,27.215300,0.00100,10000,1000,late,bicubic
10,ofw5qu7m,0.740682,30.076991,0.550809,27.124664,0.00100,10000,0,late,bilinear
5,ep3o5p3r,0.740355,30.045255,0.548691,27.059936,0.00010,10000,1000,early,nearest


### iter 25000 across all samples

In [234]:
csv_files_10k = ['run_20201128-173303.csv', 'run_20201128-173259.csv']
csv_paths = [path_results + f for f in csv_files_10k]
df_25k = pd.concat((pd.read_csv(f) for f in csv_paths))

# save_df_to_json(path_results + 'trials_best_20201121-160718.json', df_10k)

df_25k = agg_sort_df(df_25k)
df_25k

,trial_id,ssim_dc,psnr_dc,ssim_est,psnr_est,alpha_fm,num_iter,weight_method,downsamp_method
1,udt0c920,0.741091,31.408291,0.659518,29.944509,0.0001,25000,all,bilinear
0,0000_25k,0.726145,31.123009,0.651782,29.756182,0.0000,25000,all,bilinear


In [223]:
# get_1d_results(df_2k)
get_1d_results(df_10k)

alpha_fm
1e-05: ssim ~ N(0.7457, 0.003), psnr ~ N(30.2906, 0.1354), n=6
0.0001: ssim ~ N(0.7448, 0.0037), psnr ~ N(30.2221, 0.1372), n=10
0.001: ssim ~ N(0.7421, 0.0018), psnr ~ N(30.1243, 0.0686), n=4

iter_start_fm_loss
0: ssim ~ N(0.7462, 0.0034), psnr ~ N(30.2945, 0.1384), n=9
1000: ssim ~ N(0.7436, 0.0029), psnr ~ N(30.1798, 0.1053), n=8
1600: ssim ~ N(0.7423, 0.0026), psnr ~ N(30.1241, 0.101), n=3

weight_method
all: ssim ~ N(0.7458, 0.0036), psnr ~ N(30.2752, 0.1514), n=7
early: ssim ~ N(0.7452, 0.0032), psnr ~ N(30.25, 0.1271), n=6
late: ssim ~ N(0.7428, 0.0029), psnr ~ N(30.1479, 0.1003), n=7

downsamp_method
bicubic: ssim ~ N(0.7436, 0.0032), psnr ~ N(30.1816, 0.1157), n=5
bilinear: ssim ~ N(0.7449, 0.0036), psnr ~ N(30.2345, 0.141), n=9
nearest: ssim ~ N(0.7449, 0.0034), psnr ~ N(30.2406, 0.1533), n=6

